In [3]:
import numpy as np
import cv2
import dlib

In [15]:
class FeatureExtractor:
    
    def __init__(self, dlib_shape_predictor_filepath):
        
        self.dlib_shape_predictor_filepath = dlib_shape_predictor_filepath


    def extract_features(self, image_path):
        """
        Extracts features from a 2D image.

        Args:
          image: the path to the image

        Resources:
          https://www.pyimagesearch.com/2017/04/03/facial-landmarks-dlib-opencv-python/

        Returns:
          A list of extracted features in the form [x1, y1, x2, y2, ..., xn, yn]
        """

        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(self.dlib_shape_predictor_filepath)
        image = cv2.imread(image_path)
        rects = detector(image, 1)

        features_x = []
        features_y = []
        for rect in rects:
            shape = predictor(image, rect)
            for i in range(shape.num_parts):
                features_x.append(shape.part(i).x)
                features_y.append(shape.part(i).y)

        features_x = np.array(features_x)
        features_y = np.array(features_y)
        
        features_x = (features_x - np.mean(features_x)) / np.std(features_x)
        features_y = (features_y - np.mean(features_y)) / np.std(features_y)
        
        return np.reshape(np.concatenate([features_x.reshape(68,1),features_y.reshape(68,1)],axis=-1), (136))
